# QE to cif

In [1]:
using PorousMaterials, LinearAlgebra

In [2]:
xtal_name = "NiPyC2_relax.cif"
crystal = Crystal(xtal_name)
crystal.box.f_to_c

3×3 Array{Float64,2}:
 6.2528   7.66837e-16  -0.227594   
 0.0     12.5234        6.43209e-16
 0.0      0.0          10.2743     

In [3]:
function find_start_coords(lines::Array{String,1})
    start_coords = 0
    for line in lines
        start_coords += 1
        if occursin("Begin final coordinates", line)
            return start_coords + 2
        end
    end
end

function find_no_atoms(lines::Array{String,1})
    for line in lines
        if occursin("number of atoms/cell", line)
            return parse(Int64, split(line)[5])
        end
    end
end

function read_coords(lines::Array{String,1})
    n_atoms = find_no_atoms(lines)
    xf = zeros(3, n_atoms)
    species = [:blah for i = 1:n_atoms]
    start_coords = find_start_coords(lines)
    for a = 1:n_atoms
        line = split(lines[start_coords+a])
        species[a] = Symbol(line[1])
        for c = 1:3
            xf[c, a] = parse(Float64, line[1+c])
        end
    end
    return species, xf
end

function read_lattice_parameter(lines::Array{String,1})
    for line in lines
        if occursin("lattice parameter", line)
            # Bohr to Angstrom
            return parse(Float64, split(line)[5]) * 0.529177
        end
    end
end

function read_box(lines::Array{String,1})
    # find start of box info
    box_start = 0
    for line in lines
        box_start += 1
        if occursin("crystal axes", line)
            break
        end
    end
    f_to_c = zeros(3, 3)
    for i = 1:3
        for j = 1:3
            f_to_c[i, j] = parse(Float64, split(lines[box_start+j])[3+i])
        end
    end
    
    # scale by lattice params
    return f_to_c * read_lattice_parameter(lines)
end

function read_unitcell_volume(lines::Array{String,1})
    for line in lines
        if occursin("unit-cell volume", line)
            return parse(Float64, split(line)[4]) * 0.529177 ^ 3 # Bohr to Angstrom
        end
    end
end

function qe_output_to_cif(qe_output_filename::String)
    # read in lines of QE output file
    qe_file = open(qe_output_filename)
    lines = readlines(qe_file)
    close(qe_file)
    
    # read species and coords
    species, xf = read_coords(lines)
    atoms = Atoms(species, Frac(xf))

    # read unit cell info
    f_to_c = read_box(lines)
    box = Box(f_to_c)
    
    # assert unit cell volume same as determinant of f to c matrix
    @assert isapprox(read_unitcell_volume(lines), det(f_to_c), atol=0.2)

    return Crystal(qe_output_filename, box, atoms, Charges{Frac}(0))
end

qe_output_filename = joinpath("structural_relaxation", "pw_NiPyC2_P1.relax.out")
xtal = qe_output_to_cif(qe_output_filename)

write_cif(crystal, "Nick.cif")
xtal.box

Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 91.269007 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 6.252808 Å. b = 12.523418 Å, c = 10.276816 Å
	Volume of unit cell: 804.544507 Å³


In [4]:
crystal.box

Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 91.269000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 6.252800 Å. b = 12.523400 Å, c = 10.276800 Å
	Volume of unit cell: 804.540972 Å³
